# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. This is the dataset to use for this assignment. Note: The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [1]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

In [94]:
df = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')
df['Date'] = pd.to_datetime(df['Date']) #convert date values to datetime
df.sort_values(by=['Date'], inplace= True) #sort data by date
df['Data_Value'] = df['Data_Value']/10 # convert data to degrees C
df['Year'] = df['Date'].dt.year #extract year from dates
df['Month-Day'] = df['Date'].dt.strftime('%m-%d') #extract month/day from datetime
df = df[df['Month-Day'] != '02-29'] #eliminate leap days from data
highs_05to14 = df[(df['Year'] < 2015) & (df['Element'] == 'TMAX')].groupby('Month-Day').max()
highs_05to14.shape

(365, 5)

In [95]:
lows_05to14 = df[(df['Year'] < 2015) & (df['Element'] == 'TMIN')].groupby('Month-Day').min()
lows_05to14['Data_Value'].shape

(365,)

In [97]:
data2015 = df[~(df['Year'] < 2015)]
highs_2015 = data2015[data2015['Element'] == 'TMAX'].groupby('Month-Day').max()
lows_2015 = data2015[data2015['Element'] == 'TMIN'].groupby('Month-Day').min()
data2015.head()

,ID,Date,Element,Data_Value,Year,Month-Day
43095,USW00014833,2015-01-01,TMIN,-8.8,2015,01-01
51805,USC00208972,2015-01-01,TMAX,-6.7,2015,01-01
43140,USC00202308,2015-01-01,TMIN,-10.6,2015,01-01
43653,USC00200228,2015-01-01,TMAX,-5.0,2015,01-01
43652,USC00200228,2015-01-01,TMIN,-8.9,2015,01-01


In [98]:
highs_2015.reset_index(inplace=True)
lows_2015.reset_index(inplace=True)

highs_05to14.reset_index(inplace=True)
lows_05to14.reset_index(inplace=True)

record_highs_2015 = highs_2015[highs_2015['Data_Value'] > highs_05to14['Data_Value']]
record_lows_2015 = lows_2015[lows_2015['Data_Value'] < lows_05to14['Data_Value']]
record_lows_2015.head()

,Month-Day,ID,Date,Element,Data_Value,Year
4,01-05,USC00200032,2015-01-05,TMIN,-15.5,2015
10,01-11,USC00200032,2015-01-11,TMIN,-20.0,2015
33,02-03,USC00200032,2015-02-03,TMIN,-23.8,2015
44,02-14,USC00200032,2015-02-14,TMIN,-23.9,2015
45,02-15,USC00200032,2015-02-15,TMIN,-26.0,2015


In [108]:
highs_10yr = highs_05to14['Data_Value']
lows_10yr = lows_05to14['Data_Value']

import numpy as np
import matplotlib.dates as dates
import matplotlib.ticker as ticker

#create x-axis to span a single year
chart_dates = np.arange('2015-01-01', '2016-01-01', dtype='datetime64[D]')
#create a new figure
plt.figure()

#plot highs and lows
plt.scatter(record_lows_2015.Date.values,record_lows_2015.Data_Value.values,s=12,color='blue')
plt.plot(chart_dates,highs_10yr,'-',color='tomato')
plt.plot(chart_dates,lows_10yr,'-',color='mediumslateblue')
plt.scatter(record_highs_2015.Date.values,record_highs_2015.Data_Value.values,s=12,color='red')

#format 
ax = plt.gca()
ax.axis(['2015/01/01','2015/12/31',-50,50])
plt.xlabel('Date', fontsize=12)
plt.ylabel('$^\circ$C', fontsize=14).set_rotation(0)
plt.title('Record Temps around Ann Arbor, MI (2005-2015)', fontsize=14)
ax.fill_between(chart_dates,lows_10yr, highs_10yr, facecolor='blue', alpha=0.1)
plt.legend(['10 Year Highs 2005-2014','10 Year Lows 2005-2014','Record Breaking Highs 2015','Record Breaking Lows 2015'],frameon=False)


#clean up x axis tick labels
ax.xaxis.set_major_formatter(ticker.NullFormatter())
ax.xaxis.set_minor_formatter(dates.DateFormatter('%b'))

ax.xaxis.set_major_locator(dates.MonthLocator())
ax.xaxis.set_minor_locator(dates.MonthLocator(bymonthday=15))

#hide minor ticks
ax.tick_params(which='minor', width=0)

#create function to convert degrees celsius to degrees fahrenheit
def c_to_f(temp):
    """
    Returns temperature in Fahrenheit given Celsius temperature.
    """
    return (temp * 9. / 5.) + 32
#
ax2 = ax.twinx()
ax2.set_ylabel('$^\circ$F', fontsize=14).set_rotation(0)
y1,y2 = ax.get_ylim()
ax2.set_ylim(c_to_f(y1),c_to_f(y2))
ax2.yaxis.set_major_locator(ticker.MultipleLocator(30))
ax2.set_yticks(ax2.get_yticks()[1:-2])


plt.show()